In [1]:
import argparse
import copy
import json
import os
import sys

try:
    import apex
except:
    print("No APEX!")
import numpy as np
import torch
from torch import nn
import yaml
from det3d import torchie
from det3d.datasets import build_dataloader, build_dataset
from det3d.models import build_detector
from det3d.torchie import Config
from det3d.torchie.apis import (
    batch_processor,
    build_optimizer,
    get_root_logger,
    init_dist,
    set_random_seed,
    train_detector,
    example_to_device,
)
from det3d.torchie.trainer import load_checkpoint
import pickle 
import time 
from matplotlib import pyplot as plt 
from det3d.torchie.parallel import collate, collate_kitti
from torch.utils.data import DataLoader
import matplotlib.cm as cm
import subprocess
import cv2
from tools.demo_utils import visual 
from collections import defaultdict

import glob

No APEX!
No Tensorflow
Deformable Convolution not built!
No APEX!


In [2]:
def convert_box(info):
    boxes =  info["gt_boxes"].astype(np.float32)
    names = info["gt_names"]

    assert len(boxes) == len(names)

    detection = {}

    detection['box3d_lidar'] = boxes

    # dummy value 
    detection['label_preds'] = np.zeros(len(boxes)) 
    detection['scores'] = np.ones(len(boxes))

    return detection

In [3]:
cfg = Config.fromfile('configs/nusc/pp/nusc_centerpoint_pp_02voxel_two_pfn_10sweep_demo.py')

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)

dataset = build_dataset(cfg.data.val)

data_loader = DataLoader(
    dataset,
    batch_size=1,
    sampler=None,
    shuffle=False,
    num_workers=8,
    collate_fn=collate_kitti,
    pin_memory=False,
)

checkpoint = load_checkpoint(model, './latest.pth', map_location="cpu")
model.eval()

model = model.cuda()

gpu_device = torch.device("cuda")

points_list = [] 
gt_annos = [] 
detections  = [] 

data_iter = iter(data_loader)


Use HM Bias:  -2.19
10


In [4]:
data_batch = next(data_iter)

In [5]:
# save the input points for tensorrt
data_batch["points"].cpu().numpy()[:,1:].astype(np.float32).tofile("./tensorrt/data/centerpoint/points.bin")

In [6]:
example = example_to_device(data_batch, gpu_device, non_blocking=False)

# Visualize Pytorch Results

In [7]:
# do inference by pytroch and visualiza the results to "demo/torch_demo"
outputs = model(example, return_loss=False)

In [8]:
gt_annos = [] 
detections  = []
points_list = []
cpu_device = torch.device("cpu")
i = 0
info = dataset._nusc_infos[i]
gt_annos.append(convert_box(info))

points = data_batch['points'][:, 1:4].cpu().numpy()
for output in outputs:
    for k, v in output.items():
        if k not in [
            "metadata",
        ]:
            output[k] = v.to(cpu_device)
    detections.append(output)

points_list.append(points.T)

print('Done model inference. Please wait a minute, the matplotlib is a little slow...')

for i in range(len(points_list)):
    visual(points_list[i], gt_annos[i], detections[i], i, save_path="demo/torch_demo")
    print("Rendered Image {}".format(i))

Done model inference. Please wait a minute, the matplotlib is a little slow...
Rendered Image 0


# Visualize TensorRT Result
1. copy the ./tensorrt/data/centerpoint/points.bin to <TensorRT root directory\>/data/centerpoint
2. run the <TensorRT root directory\>/bin/centerpoint to get the tensorrt outputs.
3. copy the <TensorRT root directory\>/data/centerpoint back the CenterPoint/tensorrt/data
4. run the following python code to visualiza tensorrt result

In [12]:
keys = ['reg', 'height', 'dim', 'rot', 'vel', 'hm']
trtFiles = glob.glob("./tensorrt/data/centerpoint/*")
trt_input = sorted(trtFiles)[:36]

trt_pred = []
for task in range(6):
    task_pred = {}
    for idx,key in enumerate(keys):
        trt_res = np.fromfile(trt_input[task*6+idx],np.float32).reshape(1,-1,128,128)
        task_pred[key] = torch.from_numpy(trt_res).cuda()
    trt_pred.append(task_pred)

In [13]:
outputs = model.bbox_head.predict(example, trt_pred, model.test_cfg)

In [14]:
gt_annos = [] 
detections  = []
points_list = []
cpu_device = torch.device("cpu")
i = 0
info = dataset._nusc_infos[i]
gt_annos.append(convert_box(info))

points = data_batch['points'][:, 1:4].cpu().numpy()
for output in outputs:
    for k, v in output.items():
        if k not in [
            "metadata",
        ]:
            output[k] = v.to(cpu_device)
    detections.append(output)

points_list.append(points.T)

print('Done model inference. Please wait a minute, the matplotlib is a little slow...')

for i in range(len(points_list)):
    visual(points_list[i], gt_annos[i], detections[i], i, save_path="demo/trt_demo")
    print("Rendered Image {}".format(i))

Done model inference. Please wait a minute, the matplotlib is a little slow...
Rendered Image 0
